In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

from environment.env import ObservableDeformedGridworld
import numpy as np 
import torch

# create dataset

In [2]:
from tqdm import trange
env = ObservableDeformedGridworld(
    stretch=(1, 1),
    shear=(.0, .0),
    render_mode="human"
)

num_positions = 1000
num_defomations = 1000
dataset = []

for i in trange(num_positions):
    pos = env.set_pos_nodeform()
    obstacle = env.is_in_obstacle_nodeform(pos)

    for _ in range(num_defomations):
        env.set_deformation(env.sample(2,env.stretch_range), env.sample(2,env.shear_range))
        defomed_obstacle = env.is_in_obstacle(pos)
        
        datapoint = {"pos":       pos,
                    "theta":      env.transformation_matrix,
                    "obs":        1 if obstacle else 0,
                    "deform_obs": 1 if defomed_obstacle else 0,
                }
        # env.render()
        dataset.append(datapoint)


100%|██████████| 1000/1000 [15:16<00:00,  1.09it/s]


In [3]:
len(dataset)

1000000

In [4]:
# save dataset
import pickle

with open("dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)

# train


In [1]:
import pickle
# lod dataset
with open("dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

print(len(dataset))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.pkl'

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, data_list):
        """
        Args:
            data_list (list): List of dictionaries containing 'o', 'theta', 'qpos', and 'qpos_new'.
        """
        self.data_list = data_list

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.data_list)

    def __getitem__(self, idx):
        """
        Retrieve one sample of data by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            A dictionary with inputs and expected outputs as tensors.
        """
        # Extract the dictionary for the given index
        data = self.data_list[idx]
        
        # Convert data to PyTorch tensors
        o = torch.tensor(data['obs'], dtype=torch.float32).unsqueeze(0)
        theta = torch.tensor(data['theta'], dtype=torch.float32).flatten()
        o_new = torch.tensor(data['deform_obs'], dtype=torch.float32).unsqueeze(0)
        pos = torch.tensor(data['pos'], dtype=torch.float32)      
        

        # Inputs: qpos_new, o, theta
        inputs = {
            'deform_obs': o_new,
            'theta': theta,
            'pos': pos
        }
        
        # Output: qpos_new
        target = {
            'pos': pos,
            'obs': o
        }
        
        return inputs, target


# Instantiate the dataset
custom_dataset = CustomDataset(dataset)

# Create a DataLoader
data_loader = DataLoader(custom_dataset, batch_size=1024, shuffle=True)


In [7]:
for inputs, target in data_loader:
    print((inputs['deform_obs'].shape,inputs['theta'].shape))
    break


(torch.Size([1024, 1]), torch.Size([1024, 4]))


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.f1 = nn.Linear(7, 128)
        self.f2 = nn.Linear(128, 128)
        self.f3 = nn.Linear(128, 128)
        self.f4 = nn.Linear(128, 1)
        
    def forward(self, pos,deform_obs,theta):
        x = torch.cat([pos,deform_obs,theta], dim=1)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = F.sigmoid(self.f4(x))
        return x

# Instantiate the model
model = NN()

# Check the output size
# Iterate through the DataLoader
for inputs, target in data_loader:
    print(model(inputs['pos'],inputs['deform_obs'],inputs['theta']).shape)
    break




torch.Size([1024, 1])


In [9]:
# train network 
import torch.optim as optim
from tqdm.notebook import tqdm
# Instantiate the model
model = NN()


# Define the loss function
criterion = nn.BCELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the model in training mode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()



NN(
  (f1): Linear(in_features=7, out_features=128, bias=True)
  (f2): Linear(in_features=128, out_features=128, bias=True)
  (f3): Linear(in_features=128, out_features=128, bias=True)
  (f4): Linear(in_features=128, out_features=1, bias=True)
)

In [10]:
# tqdm progress bar
pbar = tqdm(total=len(data_loader),desc="Training")
pbar.refresh()
pbar.reset()

# Iterate through the DataLoader
for epoch in range(25):
    running_loss = 0.0
    pbar.set_description(f"Epoch {epoch}")
    for i, (inputs, target) in enumerate(data_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs['pos'].to(device),inputs['deform_obs'].to(device),inputs['theta'].to(device))

        # Compute the loss
        loss = criterion(outputs, target['obs'].to(device))

        # Backward pass
        running_loss += loss.item()
        loss.backward()

        # Update the weights
        optimizer.step()

        pbar.update(1)
    pbar.reset()
    print("runningLoss:", running_loss/len(data_loader))


Training:   0%|          | 0/977 [00:00<?, ?it/s]

runningLoss: 0.2144535858653097
runningLoss: 0.08388630141377328
runningLoss: 0.039968182678842494
runningLoss: 0.025597751540958332
runningLoss: 0.019215802113480767
runningLoss: 0.01612439852716933
runningLoss: 0.012459583046198836
runningLoss: 0.011921922532801527
runningLoss: 0.010664185627764939
runningLoss: 0.009455553987235053
runningLoss: 0.007602920692114726
runningLoss: 0.0078061302932467365
runningLoss: 0.007648480576610797
runningLoss: 0.008177641274797384
runningLoss: 0.006873367383943617
runningLoss: 0.007289401397462686
runningLoss: 0.006344022617404642
runningLoss: 0.006441807834419137
runningLoss: 0.005146051003853147
runningLoss: 0.006723761171156859
runningLoss: 0.006416656239019724
runningLoss: 0.004835184061141473
runningLoss: 0.005166999004807937
runningLoss: 0.005838286008353621
runningLoss: 0.004377752987125997


In [11]:
# save model
torch.save(model.state_dict(), "obs_model.pth")

In [ ]:
# filter target with obstacle
count = 0
for inputs, target in data_loader:
    count += torch.count_nonzero(target['obs'])


In [ ]:
len(dataset), count.item(), count.item()/len(dataset) 

(1000000, 121000, 0.121)

In [ ]:
count

tensor(121000)

In [ ]:
# test model
model.eval()
with torch.no_grad():
    for inputs, target in data_loader:
        outputs = model(inputs['pos'].to(device),inputs['deform_obs'].to(device),inputs['theta'].to(device))
        print(inputs)
        print(outputs)
        break

{'deform_obs': tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]), 'theta': tensor([[ 0.6069,  0.0460, -0.1410,  0.7324],
        [ 0.4951, -0.0290, -0.1900,  0.5352],
        [ 0.5310, -0.0502,  0.1989,  0.4344],
        ...,
        [ 0.5591, -0.1723,  0.1204,  0.5878],
        [ 0.9620,  0.1982, -0.0428,  0.7177],
        [ 0.4606, -0.0163,  0.0165,  0.8431]]), 'pos': tensor([[0.7033, 0.4231],
        [0.5368, 0.4076],
        [1.1467, 1.1071],
        ...,
        [0.4275, 1.1039],
        [1.0132, 1.0841],
        [0.0238, 1.0522]])}
tensor([[4.3709e-04],
        [4.6240e-01],
        [6.8391e-05],
        ...,
        [4.8210e-02],
        [1.5263e-03],
        [1.8719e-02]], device='cuda:0')


In [ ]:
outputs[outputs>0.5].shape

torch.Size([94])